# Aplikasi Sampling Penutup Lahan
## Bagian 2. Sampling tool (versi 2)

2021-08-23

Tool untuk assign class pada setiap titik sampel. Perbaikan dari versi sebelumnya dengan menambah fitur `interact_manual` dan juga opsi untuk menggunakan `Dropdown` yang lebih praktis dalam proses pengisian atribut yang tidak dilakukan sekali jalan. Pada versi ini juga ditambahkan fiture pemilihan ID dengan menggunakan `widgets.Button`.

In [ ]:
import geopandas
from ipywidgets import widgets, interact, interactive, fixed, interact_manual
from IPython.display import display, clear_output
from ipyleaflet import WidgetControl
import ipyleaflet
import ee
import geemap
import numpy

In [ ]:
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_fetcher = map_id['tile_fetcher']
  return tile_fetcher.url_format

In [ ]:
gdf = geopandas.read_file('./samples/Aug-06-2021_Borneo_2016_stratifiedsamples_15_1000.shp')
gdf['ID'] = numpy.arange(len(gdf))
gdf['lat'] = gdf['geometry'].y
gdf['lon'] = gdf['geometry'].x
gdf['Class'] = None
id_list = gdf['ID'].tolist()

landsat = ee.Image('users/kfaisal/LCMS_Borneo_2016/L8_Borneo_2016_int')

### using `widgets.BoundedIntText` to select ID

Metode ini ideal digunakan untuk pengisian atribut secara kontinyu. Pengisian dilakukan **satu persatu** dan **berurutan**. Pemilihan objek dilakukan berdasarkan ID. Id bisa dipilih melalui widget `BoundedIntText` dengan meng-klik tombol up atau down ataupun dengan mengetikkan angka OID. Pengisian kelas dilakukan dengan memilih tipe penutup lahan pada dropdown list dan kemudian meng-klik `Run Interact` untuk meng-update perubahan pada file geodataframe. Jika `Run Interact` tidak di-klik, maka perubahan tidak akan terjadi. Ini fitur yang membedakan dengan versi sebelumnya.

Isu: jika menggunakan `ipyleaflet.WidgetControl` untuk menampilkan toolbar pada badan peta, fitur `Run Interact` tidak muncul. Sehingga untuk sementara `ipyleaflet.WidgetControl` tidak digunakan yang membuat estetika GUI sedikit berkurang.

In [ ]:
# widgets
oid_selector = widgets.BoundedIntText(0,0,1000,1, description = 'Select ID')
class_assign = widgets.Dropdown(options = ['Forest','No-forest'],
                                description = 'Class name:',
                                value = None)
out = widgets.Output()
toolbar_widget = widgets.VBox([class_assign, out, oid_selector])
toolbar_ctrl = WidgetControl(widget = toolbar_widget, position = "topleft")

# interact
def filt_row(oid):
    map = geemap.Map()
    gdf_obj = gdf.loc[gdf['ID'] == oid]
    ee_obj = geemap.geopandas_to_ee(gdf_obj)
    map.addLayer(landsat, {'bands': ['B5','B6','B7'],
                           'min': 40,
                           'max': 4000},
                'Landsat image')
    map.addLayer(ee_obj)
    # map.add_control(toolbar_ctrl)
    map.centerObject(ee_obj, 10)
    # print('Selected ID: {}'.format(oid))
    print('Selected ID and Class name: {}'.format(gdf.loc[gdf['ID'] == oid, 'Class']))
    return map

def assign_f(classname):
    oid = oid_selector.value
    gdf.loc[gdf['ID'] == oid, 'Class'] = classname
    print(gdf.loc[gdf['ID'] == oid])

widgets.interact_manual(assign_f, classname = class_assign, gdf = widgets.fixed(gdf))
widgets.interact(filt_row, oid = oid_selector)
# toolbar_wid = widgets.HBox([oid_selector, class_assign])
# display(toolbar_wid)

In [ ]:
# Cek hasil pengisian kelas
gdf.head(25)

### using `widgets.Dropdown` to select id

Metode ini menggunakan `Widgets.Dropdown` untuk memilih ID objek yang akan di-edit. Metode ini lebih efisien dibandingkan dengan `widgets.BoundedIntText` yang mengurutkan satu demi satu. Pada metode ini yang ditampilkan adalah list ID. Ini akan memudahkan ketika nantinya editing dilakukan tidak dalam sekali jalan. Pada pengisian berikutnya, dapat di-filter ID mana saja yang belum diisi, dan jikapun ada ID yang terlewat pengisiannya sebelumnya tidak akan menjadi masalah karena ID tidak diurutkan n+1 tapi berdasarkan posisi row (indeks).

In [ ]:
# widgets
oid_selector = widgets.Dropdown(options = id_list,
                                description = 'Select ID:',
                                value = id_list[0])
class_assign = widgets.Dropdown(options = ['Forest','No-forest'],
                                description = 'Class name:',
                                value = None)

# interact
def filt_row(oid):
    map = geemap.Map()
    gdf_obj = gdf.loc[gdf['ID'] == oid]
    ee_obj = geemap.geopandas_to_ee(gdf_obj)
    map.addLayer(landsat, {'bands': ['B5','B6','B7'],
                           'min': 40,
                           'max': 4000},
                'Landsat image')
    map.addLayer(ee_obj)
    # map.add_control(toolbar_ctrl)
    map.centerObject(ee_obj, 10)
    # print('Selected ID: {}'.format(oid))
    print('Selected ID and Class name: {}'.format(gdf.loc[gdf['ID'] == oid, 'Class']))
    return map

def assign_f(classname):
    oid = oid_selector.value
    gdf.loc[gdf['ID'] == oid, 'Class'] = classname
    print(gdf.loc[gdf['ID'] == oid])

widgets.interact_manual(assign_f, classname = class_assign, gdf = widgets.fixed(gdf))
widgets.interact(filt_row, oid = oid_selector)

In [ ]:
gdf.head(25)

In [ ]:
# export current results
from datetime import date
today = date.today()
todaydate = today.strftime("%b-%d-%Y")
exportName = 'Borneo'
suffix = 'v1'

gdf.to_file('./temp_results/' + todaydate + '_samples_' + exportName + '_' + suffix + '.shp')

### using `widgets.Button`

Metode ini menggunakan `widgets.Button` untuk memilih objek berdasarkan ID. Ini mungkin metode yang paling ramah bagi pengguna.

In [ ]:
widget_width = "250px"
padding = "0px 0px 0px 4px"  # upper, right, bottom, left

In [ ]:
initButton = widgets.Button(
    description = 'Initial feature',
    button_style = 'info',
    tooltip = 'Initial feature',
    icon = 'toggle-on'
)

showButton = widgets.Button(
    description = 'Show map!',
    button_style = 'info',
    tooltip = 'Show map!',
    icon = 'toggle-on'
)

nextButton = widgets.Button(
    description = 'Next feature',
    button_style = 'info',
    tooltip = 'Next feature',
    icon = 'toggle-right'
)

prevButton = widgets.Button(
    description = 'Prev. feature',
    button_style = 'info',
    tooltip = 'Prev. feature',
    icon = 'toggle-left'
)

toolbar_widget = widgets.VBox()
toolbar_widget.children = [
    widgets.HBox([initButton, prevButton, nextButton])
]

toolbar_ctrl = WidgetControl(widget = toolbar_widget, padding = padding, widget_width = widget_width, position = "topright")
display(toolbar_widget)

In [ ]:
out = widgets.Output()
class_assign = widgets.Dropdown(options = ['Forest','No-forest'],
                                description = 'Class name:',
                                value = None)

idx = 0 # initial ID
geodata = ipyleaflet.GeoData(geo_dataframe = gdf.loc[gdf['ID'] == 0], name = 'Layer 0')

m = ipyleaflet.Map(scroll_wheel_zoom = True)
m.center = (0, 115)
m.zoom = 7
control = ipyleaflet.LayersControl(position='topright')
m.add_control(control)
m.add_layer(ipyleaflet.TileLayer(url = GetTileLayerUrl(
    landsat.visualize(min = 400, max = 4000, gamma = 1.5, bands = ['B5','B6','B7']))))
m.add_layer(geodata)

def next_button_clicked(b, incr = 1):
    with out:
        clear_output()
        global idx
        global geodata
        print('prev id:', idx)
        idx = idx + incr
        print('current id:', idx)
        m.remove_layer(geodata)
        geodata = ipyleaflet.GeoData(geo_dataframe = gdf.loc[gdf['ID'] == id_list[idx]], name = 'Layer ' + str(id_list[idx]))
        print(geodata)
        m.add_layer(geodata)
        sel = gdf.loc[gdf['ID'] == id_list[idx]]
        lat = sel.iat[0,3]
        lon = sel.iat[0,4]
        m.center = (lat, lon)
nextButton.on_click(next_button_clicked)

def prev_button_clicked(b):
    return next_button_clicked(b, -1)

prevButton.on_click(prev_button_clicked)

def assign_f(classname):
    global idx
    gdf.loc[gdf['ID'] == idx, 'Class'] = classname
    print(gdf.loc[gdf['ID'] == idx])

display(out, widgets.HBox([prevButton, nextButton]))
interact_manual(assign_f, classname = class_assign, gdf = widgets.fixed(gdf))

m


In [ ]:
gdf.head()

## Melanjutkan proses editing

In [ ]:
# memanggil file hasil pengisian kelas yang telah dilakukan sebelumnya
gdf_v1 = geopandas.read_file('./temp_results/Aug-22-2021_samples_Borneo_v1.shp')

# filter rows yang belum diisi kelasnya
filt_gdf_v1 = gdf_v1[gdf_v1['Class'].isna()] 
filt_id_list = filt_gdf_v1['ID'].tolist()
filt_gdf_v1.head()

In [ ]:
# widgets
oid_selector = widgets.Dropdown(options = filt_id_list,
                                description = 'Select ID:',
                                value = filt_id_list[0])
class_assign = widgets.Dropdown(options = ['Forest','No-forest'],
                                description = 'Class name:',
                                value = None)

# interact
def filt_row(oid):
    map = geemap.Map()
    gdf_obj = gdf.loc[gdf['ID'] == oid]
    ee_obj = geemap.geopandas_to_ee(gdf_obj)
    map.addLayer(landsat, {'bands': ['B5','B6','B7'],
                           'min': 40,
                           'max': 4000},
                'Landsat image')
    map.addLayer(ee_obj)
    # map.add_control(toolbar_ctrl)
    map.centerObject(ee_obj, 10)
    # print('Selected ID: {}'.format(oid))
    print('Selected ID and Class name: {}'.format(gdf.loc[gdf['ID'] == oid, 'Class']))
    return map

def assign_f(classname):
    oid = oid_selector.value
    gdf.loc[gdf['ID'] == oid, 'Class'] = classname
    print(gdf.loc[gdf['ID'] == oid])

widgets.interact_manual(assign_f, classname = class_assign, gdf = widgets.fixed(gdf))
widgets.interact(filt_row, oid = oid_selector)

In [ ]:
gdf.head()

In [ ]:
# export current results
from datetime import date
today = date.today()
todaydate = today.strftime("%b-%d-%Y")
exportName = 'Borneo'
suffix = 'v2'

gdf.to_file('./temp_results/' + todaydate + '_samples_' + exportName + '_' + suffix + '.shp')

In [ ]:
# Export to Asset, later used as machine-learning classification input in GEE
from datetime import date
today = date.today()
todaydate = today.strftime("%b-%d-%Y")

# define output file name
exportName = 'Borneo'

# Convert geodataframe to ee object
ee_export = geemap.geopandas_to_ee(randsamp)

exportTask = ee.batch.Export.table.toAsset(
    collection = ee_export,
    description = todaydate + '_' + exportName + '_forestCoverSamples',
    assetId = 'users/gemasaktiadzan/2021-08-08_Borneo_forestCoverSamples')
exportTask.start()